## Lyft Baseline
The following notebook performs evaluation using a simple baseline.
The baseline has been **trained for 100k iterations with batch size 64 and history_num_frames 10** on the `train.zarr` dataset. All other parameters have been set to their default values as in [the original training configuration](https://github.com/lyft/l5kit/blob/3e3403b4d85fb99e7068cdffd0cd01d3f0d83138/examples/agent_motion_prediction/agent_motion_config.yaml)

**Note:** The notebook has been updated to work with L5Kit 1.1.0 (already available in Kaggle)


In [1]:
#!jupyter nbconvert --version
#!papermill --version

In [1]:
# ensure version of L5Kit
import l5kit
assert l5kit.__version__ == "1.1.0"

In [2]:
import numpy as np
import os
import torch

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.geometry import transform_points
from l5kit.dataset import AgentDataset
from l5kit.evaluation import write_pred_csv
from l5kit.rasterization import build_rasterizer

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50
from typing import Dict
from tqdm import tqdm

> ## Build Baseline Model

In [3]:
def build_model(cfg: Dict) -> torch.nn.Module:
    # load pre-trained Conv2D model
    model = resnet50(pretrained=False)

    # change input channels number to match the rasterizer's output
    num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
    num_in_channels = 3 + num_history_channels
    model.conv1 = nn.Conv2d(
        num_in_channels,
        model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=False,
    )
    # change output size to (X, Y) * number of future states
    num_targets = 2 * cfg["model_params"]["future_num_frames"]
    model.fc = nn.Linear(in_features=2048, out_features=num_targets)

    return model

def forward(data, model, device):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    outputs = model(inputs).reshape(targets.shape)
    return outputs

## Set the Configuration for the experiment

In [8]:
os.environ["L5KIT_DATA_FOLDER"] = "lyft-motion-prediction-autonomous-vehicles"
dm = LocalDataManager(None)

cfg = {
    'format_version': 4,
    'model_params': {
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 12,
        'shuffle': False,
        'num_workers': 0
    }

}

## Build Dataset (with mask) and Dataloader

In [9]:
# ===== INIT DATASET
test_cfg = cfg["test_data_loader"]

test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load("lyft-motion-prediction-autonomous-vehicles/scenes/mask.npz")["arr_0"]

rasterizer = build_rasterizer(cfg, dm)
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])
print(test_dataset)

/home/wabco/miniconda3/envs/lyft/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: you're running with a custom agents_mask
  


+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   11314    |  1131400   |  88594921  |    7854144    |      31.43      |        100.00        |        78.31         |        10.00         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


## Instantiate the Model and Load Weights

In [10]:
# ==== INIT MODEL
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = build_model(cfg).to(device)

model.load_state_dict(torch.load("baseline_weights.pth", map_location=device))

In [11]:
device

device(type='cuda', index=0)

### Ensure coordinates are stored in the correct reference system
Because predictions are now (from l5kit 1.1.0) in `agent` space we need to convert them back to `world` displacements before storing them.

In [12]:
# ==== EVAL LOOP
model.eval()
torch.set_grad_enabled(False)

# store information for evaluation
future_coords_offsets_pd = []
timestamps = []
agent_ids = []

progress_bar = tqdm(test_dataloader)
for data in progress_bar:
    
    # convert agent coordinates into world offsets
    agents_coords = forward(data, model, device).cpu().numpy().copy()
    world_from_agents = data["world_from_agent"].numpy()
    centroids = data["centroid"].numpy()
    coords_offset = []
    
    for agent_coords, world_from_agent, centroid in zip(agents_coords, world_from_agents, centroids):
        coords_offset.append(transform_points(agent_coords, world_from_agent) - centroid[:2])
    
    future_coords_offsets_pd.append(np.stack(coords_offset))
    timestamps.append(data["timestamp"].numpy().copy())
    agent_ids.append(data["track_id"].numpy().copy())

  0%|          | 0/5927 [00:00<?, ?it/s]/home/wabco/miniconda3/envs/lyft/lib/python3.6/site-packages/l5kit/dataset/agent.py:115: RuntimeWarning: disable_traffic_light_faces not found in config, this will raise an error in the future
  return self.get_frame(scene_index, state_index, track_id=track_id)
100%|██████████| 5927/5927 [1:38:22<00:00,  1.00it/s]


In [13]:
write_pred_csv("submission.csv",
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
              )